# Name: Abhishek Kothari
# Assignment 2A

### Question 1

Proof for KMeans Objective

Part A: E-step minimizes the objective given current centroids ($(\mu)$)

The KMeans objective is defined as:

$$
J(\pi, \mu) = min(\sum_{i=1}^n \sum_{k=1}^K \pi_{ik} \|x_i - \mu_k\|^2)
$$

Step 1: Fix centroids ($(\mu)$)
In the E-step, we update the memberships ($(\pi_{ik})$) for each data point $(x_i)$, given fixed centroids $(\mu_k)$.

Step 2: Objective for a single point
For a fixed $(x_i)$, the objective simplifies to:

$$
\sum_{k=1}^K \pi_{ik} |x_i - \mu_k|^2
$$

Here, $(\pi_{ik} \in \{0, 1\}) and (\sum_{k=1}^K \pi_{ik} = 1)$. The objective is minimized when:

$$
\pi_{ik} = 
\begin{cases} 
1 & \text{if } k = \arg\min_j \|x_i - \mu_j\|^2 \\
0 & \text{otherwise}.
\end{cases}
$$

This assigns $(x_i)$ to the closest centroid, minimizing the distance.

Conclusion:
Updating $(\pi_{ik})$ as described minimizes the objective for fixed centroids.

---

Part B: M-step minimizes the objective given current memberships ($(\pi)$)

Step 1: Fix memberships ($(\pi_{ik})$)
In the M-step, we update the centroids ($(\mu_k)$) for each cluster $(k)$, given fixed memberships $(\pi_{ik})$.

Step 2: Objective for a single centroid
For a fixed cluster $(k)$, the objective simplifies to:

$$
\sum_{i: \pi_{ik}=1} \|x_i - \mu_k\|^2
$$

This is a sum of squared distances between the centroid $(\mu_k)$ and the points assigned to cluster \(k\).

Step 3:
The optimal $(\mu_k)$ is the point that minimizes the sum of squared distances. This is the mean of the points in cluster $(k)$:

$$
\mu_k = \frac{\sum_{i: \pi_{ik}=1} x_i}{\sum_{i: \pi_{ik}=1} 1}
$$

Conclusion:
Updating $(\mu_k)$ as the mean of points minimizes the objective for fixed memberships.

---

Part C: Why does KMeans converge but not necessarily to the global minimum?

Step 1: Objective decreases monotonically
Each E-step and M-step decreases the objective function or leaves it unchanged. The KMeans objective $(J(\pi, \mu))$ is non-negative as its a distance and bounded below by 0.

Step 2: Convergence to a local minimum
Since $(J(\pi, \mu))$ decreases monotonically and is bounded, KMeans must converge to a value where further updates do not change the objective. This is typically a local minimum of the objective.

Step 3: Not guaranteed to find the global minimum
The KMeans algorithm is sensitive to the initial centroids. Poor initialization of these centroids can lead to convergence at suboptimal (local) minima.

Conclusion:
KMeans converges due to the monotonic decrease of the objective, but it does not guarantee finding the global minimum.

### Question 2

In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from collections import Counter

In [76]:
class KMeans:
    def __init__(self, n_clusters=10, max_iterations=300, metric='euclidean'):
        self.n_clusters = n_clusters
        self.max_iterations = max_iterations
        self.metric = metric
        self.centroids = None
        self.labels = None

    def fit(self, x):
        n_samples, n_features = x.shape  
    # here we first randomly assign centroids
        np.random.seed(42)
        self.centroids = x[np.random.choice(n_samples, self.n_clusters, replace=False)]
        self.labels = np.zeros(n_samples, dtype=int)  

        for _ in range(self.max_iterations):
            distances = pairwise_distances(x, self.centroids, metric=self.metric)
            new_labels = np.argmin(distances, axis=1)
            if np.all(self.labels == new_labels):
                break
            self.labels = new_labels

            for k in range(self.n_clusters):
                cluster_points = x[self.labels == k]
                if len(cluster_points) > 0:
                    self.centroids[k] = np.mean(cluster_points, axis=0)
        return self

    def inertia(self, X):
        # sum of sq dist
        distances = pairwise_distances(X, self.centroids, metric=self.metric)
        return np.sum(np.min(distances, axis=1)**2)
                
    def predict(self, X):
        # assigns labels to new data
        distances = pairwise_distances(X, self.centroids, metric=self.metric)
        return np.argmin(distances, axis=1)

    @staticmethod
    def calculate_purity_gini(labels, true_labels):
        n_samples = len(true_labels)
        clusters = np.unique(labels)
        purity, gini = 0, 0

        for cluster in clusters:
            indices = np.where(labels == cluster)[0]
            cluster_labels = true_labels[indices]
            most_common_label, count = Counter(cluster_labels).most_common(1)[0]
            purity += count

            label_counts = Counter(cluster_labels)
            gini_k = 1 - sum((count / len(indices))**2 for count in label_counts.values())
            gini += gini_k * len(indices)

        purity /= n_samples
        gini /= n_samples
        return purity, gini


Task 1) Mnist Dataset

In [38]:
import tensorflow as tf
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images / 255.0
train_images = train_images.reshape(len(train_images), -1)
k = 10
kmeans = KMeans(n_clusters=k, max_iterations=300, metric='euclidean')
kmeans.fit(train_images)

purity, gini = KMeans.calculate_purity_gini(kmeans.labels, train_labels)
print(f"The purity for mnist for k=10 is: {purity:.4f}")
print(f"The gini for mnist for k=10 is: {gini:.4f}")

inertia = kmeans.inertia(train_images)
print(f"The inertia for MNIST with k = 10 is: {inertia:.2f}")

The purity for mnist for k=10 is: 0.5286
The gini for mnist for k=10 is: 0.6100
The inertia for MNIST with k = 10 is: 2364716.07


In [43]:
k = 20
kmeans = KMeans(n_clusters=k, max_iterations=300, metric='euclidean')
kmeans.fit(train_images)

purity, gini = KMeans.calculate_purity_gini(kmeans.labels, train_labels)
print(f"The purity for mnist for k=20 is: {purity:.4f}")
print(f"The gini for mnist for k=20 is: {gini:.4f}")

inertia = kmeans.inertia(train_images)
print(f"The inertia for MNIST with k = 20 is: {inertia:.2f}")

The purity for mnist for k=20 is: 0.7091
The gini for mnist for k=20 is: 0.3974
The inertia for MNIST with k = 20 is: 2111960.40


In [45]:
k = 5
kmeans = KMeans(n_clusters=k, max_iterations=300, metric='euclidean')
kmeans.fit(train_images)

purity, gini = KMeans.calculate_purity_gini(kmeans.labels, train_labels)
print(f"The purity for mnist for k=5 is: {purity:.4f}")
print(f"The gini for mnist for k=5 is: {gini:.4f}")

inertia = kmeans.inertia(train_images)
print(f"The inertia for MNIST with k = 5 is: {inertia:.2f}")

The purity for mnist for k=5 is: 0.4523
The gini for mnist for k=5 is: 0.6585
The inertia for MNIST with k = 5 is: 2605960.41


Task 2) Fashion Dataset

In [5]:
import tensorflow as tf

In [7]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_data = np.concatenate((x_train,x_test),axis = 0)
y_data = np.concatenate((y_train,y_test), axis = 0)

In [9]:
x_data = x_data/255.0
x_data = x_data.reshape(x_data.shape[0],-1)
x_data = normalize(x_data,axis = 1)

In [11]:
print("Dataset shape:", x_data.shape)  
print("Labels shape:", y_data.shape)   

Dataset shape: (70000, 784)
Labels shape: (70000,)


In [84]:
k = 10
kmeans_f = KMeans(n_clusters = k)
kmeans_f.fit(x_data)

In [86]:
purity, gini = kmeans_f.calculate_purity_gini(kmeans_f.labels, y_data)
print(f"The purity of fashion data for k = 10 is: {purity}")
print(f"The gini of fashion data for k = 10 is: {gini}")


The purity of fashion data for k = 10 is: 0.5675428571428571
The gini of fashion data for k = 10 is: 0.5411670406959518


In [90]:
inertia = kmeans_f.inertia(x_data)
print(f"The inertia for fashion data with k = 10 is: {inertia}")

The inertia for fashion data with k = 10 is: 14931.30489011417


In [92]:
k = 20
kmeans_f = KMeans(n_clusters = k)
kmeans_f.fit(x_data)

In [93]:
purity, gini = kmeans_f.calculate_purity_gini(kmeans_f.labels, y_data)
print(f"The purity of fashion data for k = 20 is: {purity}")
print(f"The gini of fashion data for k = 20 is: {gini}")


The purity of fashion data for k = 20 is: 0.6851428571428572
The gini of fashion data for k = 20 is: 0.42128678650165496


In [96]:
inertia = kmeans_f.inertia(x_data)
print(f"The inertia for fashion data with k = 20 is: {inertia}")

The inertia for fashion data with k = 20 is: 13150.48627279295


In [98]:
k = 5
kmeans_f = KMeans(n_clusters = k)
kmeans_f.fit(x_data)

In [100]:
purity, gini = kmeans_f.calculate_purity_gini(kmeans_f.labels, y_data)
print(f"The purity of fashion data for k = 5 is: {purity}")
print(f"The gini of fashion data for k = 5 is: {gini}")


The purity of fashion data for k = 5 is: 0.3717285714285714
The gini of fashion data for k = 5 is: 0.700740594897485


In [102]:
inertia = kmeans_f.inertia(x_data)
print(f"The inertia for fashion data with k = 5 is: {inertia}")

The inertia for fashion data with k = 5 is: 17515.48710666463


Task 3) 20NG Dataset

In [6]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
nggroup = fetch_20newsgroups(subset = 'all', remove=('headers','footers','quotes'))
data, labels = nggroup.data, nggroup.target
vectorizer = TfidfVectorizer(stop_words = 'english')
x = vectorizer.fit_transform(data)

In [80]:
from sklearn.preprocessing import normalize
from collections import Counter
import numpy as np
from scipy.sparse import csr_matrix, vstack

class KMeans_ng:
    def __init__(self, n_clusters=20, max_iterations=300):
        self.n_clusters = n_clusters
        self.max_iterations = max_iterations

    def fit(self, X):
        X = normalize(X, axis=1)
        n_samples, n_features = X.shape

        # randomly init the centroids
        self.centroids = X[np.random.choice(n_samples, self.n_clusters, replace=False)].toarray()
        self.labels = np.zeros(n_samples, dtype=np.int32)

        for _ in range(self.max_iterations):
            # dot prod similarity
            similarities = X.dot(self.centroids.T)

            new_labels = np.argmax(similarities, axis=1)

            if np.all(self.labels == new_labels):
                break
            self.labels = new_labels

            for k in range(self.n_clusters):
                indices = np.where(self.labels == k)[0]

                if len(indices) > 0:
                    cluster_points = X[indices]
                    # we convert the sparse matrix in a dense row
                    self.centroids[k] = cluster_points.mean(axis=0).A1  

        return self

    def inertia(self, X):
        X = normalize(X, axis=1)
        similarities = X.dot(self.centroids.T)
        return np.sum(np.max(similarities, axis=1))

    def predict(self, X):
        X = normalize(X, axis=1)
        similarities = X.dot(self.centroids.T)
        return np.argmax(similarities, axis=1)

    @staticmethod
    def calculate_purity_gini(labels, true_labels):
        n_samples = len(true_labels)
        clusters = np.unique(labels)
        purity, gini = 0, 0

        for cluster in clusters:
            indices = np.where(labels == cluster)[0]
            cluster_labels = true_labels[indices]
            most_common_label, count = Counter(cluster_labels).most_common(1)[0]
            purity += count

            label_counts = Counter(cluster_labels)
            gini_k = 1 - sum((count / len(indices))**2 for count in label_counts.values())
            gini += gini_k * len(indices)

        purity /= n_samples
        gini /= n_samples
        return purity, gini

In [29]:
k = 20
kmeans = KMeans_ng(n_clusters = k)
kmeans.fit(x)

In [31]:
purity, gini = kmeans.calculate_purity_gini(kmeans.labels, labels)
print(f"The purity of ng group data for k = 20 is: {purity}")
print(f"The gini of ng group for k = 20 is: {gini}")


The purity of ng group data for k = 20 is: 0.426244295871803
The gini of ng group for k = 20 is: 0.7112419326681252


In [35]:
inertia = kmeans.inertia(x)
print(f"The inertia for ng group data with k = 20 is: {inertia}")

The inertia for ng group data with k = 20 is: 400.8554411190797


In [48]:
k = 40
kmeans_40 = KMeans_ng(n_clusters = k)
kmeans_40.fit(x)

In [50]:
purity, gini = kmeans_40.calculate_purity_gini(kmeans_40.labels, labels)
print(f"The purity of ng group data for k = 40 is: {purity}")
print(f"The gini of ng group for k = 40 is: {gini}")


The purity of ng group data for k = 40 is: 0.44014645017510345
The gini of ng group for k = 40 is: 0.6990602091060141


In [52]:
inertia = kmeans_40.inertia(x)
print(f"The inertia for ng group data with k = 40 is: {inertia}")

The inertia for ng group data with k = 40 is: 510.562605924934


In [54]:
k = 10
kmeans_10 = KMeans_ng(n_clusters = k)
kmeans_10.fit(x)

In [56]:
purity, gini = kmeans_10.calculate_purity_gini(kmeans_10.labels, labels)
print(f"The purity of ng group data for k = 10 is: {purity}")
print(f"The gini of ng group for k = 10 is: {gini}")


The purity of ng group data for k = 10 is: 0.2864798896317521
The gini of ng group for k = 10 is: 0.7986714606850498


In [58]:
inertia = kmeans_10.inertia(x)
print(f"The inertia for ng group data with k = 10 is: {inertia}")

The inertia for ng group data with k = 10 is: 297.07700080309326


Task 4)

In [26]:
import numpy as np
from sklearn.preprocessing import normalize
from tensorflow.keras.datasets import fashion_mnist
from sklearn.metrics import pairwise_distances

class SoftKMeans:
    def __init__(self, n_clusters=10, beta=1.0, max_iterations=100, tol=1e-4):
        self.n_clusters = n_clusters
        self.beta = beta
        self.max_iterations = max_iterations
        self.tol = tol

    def fit(self, X):
        # take random centroids
        n_samples, n_features = X.shape
        self.centroids = X[np.random.choice(n_samples, self.n_clusters, replace=False)]

        for iteration in range(self.max_iterations):
            distances = pairwise_distances(X, self.centroids, metric='euclidean')
            
            # use the soft kmeans formula
            exponentials = np.exp(-self.beta * distances)
            self.responsibilities = exponentials / np.sum(exponentials, axis=1, keepdims=True)

            # update cebntroids based on weighted means
            new_centroids = np.dot(self.responsibilities.T, X) / np.sum(self.responsibilities.T, axis=1, keepdims=True)

            # check if converged or not
            if np.linalg.norm(new_centroids - self.centroids) < self.tol:
                break
            self.centroids = new_centroids

        return self

    def predict(self, X):
        distances = pairwise_distances(X, self.centroids, metric='euclidean')
        exponentials = np.exp(-self.beta * distances)
        responsibilities = exponentials / np.sum(exponentials, axis=1, keepdims=True)
        return np.argmax(responsibilities, axis=1)

    @staticmethod
    def calc_purity_gini(labels, true_labels):
        from collections import Counter
        n_samples = len(labels)
        clusters = np.unique(labels)
        purity, gini = 0, 0

        for cluster in clusters:
            indices = np.where(labels == cluster)[0]
            cluster_labels = true_labels[indices]
            most_common_label, count = Counter(cluster_labels).most_common(1)[0]
            purity += count

            label_counts = Counter(cluster_labels)
            gini_k = 1 - sum((count / len(indices))**2 for count in label_counts.values())
            gini += gini_k * len(indices)

        purity /= n_samples
        gini /= n_samples
        return purity, gini

In [28]:
soft_kmeans = SoftKMeans(n_clusters = 10)
soft_kmeans.fit(x_data)
pred = soft_kmeans.predict(x_data)
purity, gini = soft_kmeans.calc_purity_gini(pred, y_data)
print(f"The purity and gini for the fashion dataset using soft kmeans for k = 10 is {purity}, {gini}")


The purity and gini for the fashion dataset using soft kmeans for k = 10 is 0.2583, 0.7847452538797828


### Question 3

Task 1

In [34]:
from scipy.stats import multivariate_normal

data = np.loadtxt("2gaussian.txt") 
K = 2
n, d = data.shape
# here we randomly init means, cov and mix coeff
means = np.random.rand(K, d) * (np.max(data, axis=0) - np.min(data, axis=0)) + np.min(data, axis=0)
covariances = np.array([np.eye(d) for _ in range(K)])
mixing_coefficients = np.ones(K) / K 


max_iter = 100  
tolerance = 1e-4  
log_likelihoods = []

for iteration in range(max_iter):
    # e step
    responsibilities = np.zeros((n, K))
    for k in range(K):
        responsibilities[:, k] = mixing_coefficients[k] * multivariate_normal.pdf(data, mean=means[k], cov=covariances[k])
    responsibilities /= np.sum(responsibilities, axis=1, keepdims=True)  # Normalize responsibilities

    # m step
    N_k = np.sum(responsibilities, axis=0) 
    for k in range(K):
        means[k] = np.sum(responsibilities[:, k][:, np.newaxis] * data, axis=0) / N_k[k]
        diff = data - means[k]
        covariances[k] = (responsibilities[:, k][:, np.newaxis] * diff).T @ diff / N_k[k]
        covariances[k] += 1e-6 * np.eye(d)

    mixing_coefficients = N_k / n
    log_likelihood = np.sum(
        np.log(
            np.sum(
                [mixing_coefficients[k] * multivariate_normal.pdf(data, mean=means[k], cov=covariances[k]) for k in range(K)],
                axis=0
            )
        )
    )
    log_likelihoods.append(log_likelihood)

    if iteration > 0 and abs(log_likelihoods[-1] - log_likelihoods[-2]) < tolerance:
        print(f"Converged at iteration {iteration}")
        break

print("Estimated Means:")
print(means)
print("\nEstimated Covariances:")
print(covariances)
print("\nEstimated Mixing Coefficients:")
print(mixing_coefficients)

Converged at iteration 32
Estimated Means:
[[7.01304477 3.98307506]
 [2.99393262 3.05212028]]

Estimated Covariances:
[[[0.97494052 0.49757271]
  [0.49757271 1.00123008]]

 [[1.00990629 0.02722722]
  [0.02722722 2.93797867]]]

Estimated Mixing Coefficients:
[0.66523796 0.33476204]


Task 2

In [38]:
from scipy.stats import multivariate_normal

data = np.loadtxt("3gaussian.txt") 
K = 3
n, d = data.shape
# here we randomly init means, cov and mix coeff
means = np.random.rand(K, d) * (np.max(data, axis=0) - np.min(data, axis=0)) + np.min(data, axis=0)
covariances = np.array([np.eye(d) for _ in range(K)])
mixing_coefficients = np.ones(K) / K 


max_iter = 100  
tolerance = 1e-4  
log_likelihoods = []

for iteration in range(max_iter):
    # e step
    responsibilities = np.zeros((n, K))
    for k in range(K):
        responsibilities[:, k] = mixing_coefficients[k] * multivariate_normal.pdf(data, mean=means[k], cov=covariances[k])
    responsibilities /= np.sum(responsibilities, axis=1, keepdims=True)  # Normalize responsibilities

    # m step
    N_k = np.sum(responsibilities, axis=0) 
    for k in range(K):
        means[k] = np.sum(responsibilities[:, k][:, np.newaxis] * data, axis=0) / N_k[k]
        diff = data - means[k]
        covariances[k] = (responsibilities[:, k][:, np.newaxis] * diff).T @ diff / N_k[k]
        covariances[k] += 1e-6 * np.eye(d)

    mixing_coefficients = N_k / n
    log_likelihood = np.sum(
        np.log(
            np.sum(
                [mixing_coefficients[k] * multivariate_normal.pdf(data, mean=means[k], cov=covariances[k]) for k in range(K)],
                axis=0
            )
        )
    )
    log_likelihoods.append(log_likelihood)

    if iteration > 0 and abs(log_likelihoods[-1] - log_likelihoods[-2]) < tolerance:
        print(f"Converged at iteration {iteration}")
        break

print("Estimated Means:")
print(means)
print("\nEstimated Covariances:")
print(covariances)
print("\nEstimated Mixing Coefficients:")
print(mixing_coefficients)

Converged at iteration 54
Estimated Means:
[[7.02145618 4.01540394]
 [3.03919609 3.04707698]
 [5.01147955 7.00125092]]

Estimated Covariances:
[[[0.99059269 0.50102805]
  [0.50102805 0.9956578 ]]

 [[1.02812057 0.02589962]
  [0.02589962 3.38185022]]

 [[0.98001809 0.18534817]
  [0.18534817 0.97484192]]]

Estimated Mixing Coefficients:
[0.29845025 0.20548081 0.49606894]


### Question 4

In [37]:
import numpy as np
from scipy.stats import binom

def load_data(filename):
    with open(filename, 'r') as f:
        data = np.array([[int(bit) for bit in line.strip().split()] for line in f])
    return np.sum(data, axis=1) # calculate the number of heads oer session

def em_algorithm(filename, D=20, max_iters=100, tol=1e-6):
    
    num_heads = load_data(filename)
    N = 50
    pi = np.array([1/3, 1/3, 1/3])  
    p = np.array([0.4, 0.5, 0.6])  #common initial values instead of random for optimised convergence
    
    for _ in range(max_iters):
        responsibilities = np.zeros((N, 3))

        for i in range(3):  
            likelihood = binom.pmf(num_heads, D, p[i]) 
            responsibilities[:, i] = pi[i] * likelihood  

        responsibilities /= responsibilities.sum(axis=1, keepdims=True)  # normalize
        Nk = responsibilities.sum(axis=0) #calculating resp for each coin
        pi = np.clip(Nk / N, 1e-3, 1 - 1e-3)  #make sure the values stay within 0 - 1

        for i in range(3):
            p[i] = np.sum(responsibilities[:, i] * num_heads) / (D * (Nk[i] + 1e-8)) # avoids the zero division error 

        if np.max(np.abs(pi - Nk / N)) < tol and np.max(np.abs(p - p)) < tol:
            break

    return pi, p

In [39]:
file_name = 'coin_flips_outcome.txt'
pi_est, p_est = em_algorithm(file_name)
print(f"Estimated coin selection probability: {pi_est}")
print(f"The Estimated head bias probability: {p_est}")

Estimated coin selection probability: [0.32002095 0.21899044 0.46098861]
The Estimated head bias probability: [0.28584974 0.53032459 0.74922909]
